<a href="https://colab.research.google.com/github/milvus-io/bootcamp/blob/master/bootcamp/tutorials/integration/build_RAG_with_milvus_and_deepseek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>   <a href="https://github.com/milvus-io/bootcamp/blob/master/bootcamp/tutorials/integration/build_RAG_with_milvus_and_deepseek.ipynb" target="_blank">
    <img src="https://img.shields.io/badge/View%20on%20GitHub-555555?style=flat&logo=github&logoColor=white" alt="GitHub Repository"/>

# Build RAG usando Mistral + Milvus + los documentos articles.pkl



# Instalar dependencias

In [1]:
!pip install -qU langchain langchain-huggingface faiss-cpu openai
!pip install -U langchain-community
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.4/764.4 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00


# Cargar documentos del proyecto articles y leer los archivos .txt

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pickle
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from openai import OpenAI
import os

# Carga tus documentos desde pickle (asegúrate de que la ruta es correcta)
with open("/content/drive/MyDrive/rag_project/documents.pkl", "rb") as f:
    documents = pickle.load(f)

# Extrae los textos (fragmentos)
texts = [doc["content"] for doc in documents if doc.get("content") and doc["content"].strip()]

# Inicializa modelo de embeddings HuggingFace
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Construye la base FAISS (esto puede tardar un poco si son muchos documentos)
db = FAISS.from_texts(texts, embedding_model)

# Configura tu API key para Mistral (reemplaza con tu propia clave)
os.environ["TOGETHER_API_KEY"] = "9c35a77af98dbc8d8acab94f5d7be2ad144fdf565045833fa22c2dde5d667b77"
mistral_client = OpenAI(
    api_key=os.environ["TOGETHER_API_KEY"],
    base_url="https://api.together.xyz/v1",
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
# Función para hacer la pregunta y obtener respuesta
def preguntar_a_mistral(pregunta, k=3):
    # Busca fragmentos similares
    fragmentos = db.similarity_search(pregunta, k=k)

    # Construye contexto concatenando los fragmentos
    contexto = "\n".join([f.page_content for f in fragmentos])

    # Prepara prompt para Mistral
    system_prompt = "Eres un asistente experto en analizar informacion. Responde solo con información clara y precisa basada en el contexto. "
    user_prompt = f"<context>\n{contexto}\n</context>\n<question>\n{pregunta}\n</question>"

    # Genera respuesta
    response = mistral_client.chat.completions.create(
        model="mistralai/Mistral-7B-Instruct-v0.2",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
    )
    return response.choices[0].message.content.strip()

In [4]:
pregunta = "Extrae los metadatos de cada documento en formato json"
respuesta = preguntar_a_mistral(pregunta)
print("🧠 Respuesta generada por Mistral:\n", respuesta)

🧠 Respuesta generada por Mistral:
 <answer>
[
  {
    "title": "AVAL para Gabriela María Tituaña Maldonado",
    "date": "30 de agosto del 2024",
    "issuer": "Mesía Flores Limaico",
    "issuer_position": "Presidente de la Federación de los Pueblos SAS Kichwa de la Sierra Norte del Ecuador $ CHIJALLTA FICI y su Consejo de Gobierno",
    "recipient": "Gabriela María Tituaña Maldonado",
    "recipient_id": "C.C.100445650-3",
    "nationality": "Kichwa Otavalo",
    "province": "Imbabura",
    "canton": "Otavalo",
    "parish": "El Jordán",
    "neighborhood": "Machángara",
    "reason": "Pertenencia a la Nacionalidad Kichwra del Pueblo Otavalo y participación en procesos de Fortalecimiento Organizativo e identidad cultural comunitario de pueblos y nacionalidades."
  },
  {
    "title": "CERTIFICADO DE PERTENENCIA para Kunku Kuntui David Javier",
    "date": "16 de julio del 2025",
    "issuer": "Sape'tsama Shuarma'ka Winaru",
    "issuer_position": "Presidenta",
    "recipient": "Kunku

In [7]:
!pip install streamlit pandas openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.4 MB/s eta 0:00:00


In [18]:
%%writefile app.py
import streamlit as st
import pandas as pd
from datetime import datetime
import io

# Configuración de la página
st.set_page_config(
    page_title="Consulta RAG sobre Pueblos y Nacionalidades",
    page_icon="💬",
    layout="wide",
    initial_sidebar_state="collapsed"
)

# CSS personalizado
st.markdown("""
<style>
    .main {
        background-color: #FFFFFF;
        color: #212121;
        font-family: 'Segoe UI', sans-serif;
    }

    .custom-title {
        font-size: 28px;
        font-weight: 700;
        color: #213354;
        text-align: center;
        margin-bottom: 0.3em;
    }

    .subtitulo {
        text-align: center;
        color: #525252;
        font-size: 16px;
        margin-bottom: 1em;
    }

    .stTextInput > div > div > input {
        background-color: #f2f2f2;
        color: #000;
        border: 1px solid #ccc;
        border-radius: 10px;
    }

    .stTextArea > div > div > textarea {
        background-color: #f2f2f2;
        color: #000;
        border: 1px solid #ccc;
        border-radius: 10px;
    }

    .stButton > button {
        background-color: #213354;
        color: white;
        font-weight: bold;
        border-radius: 8px;
        padding: 10px 18px;
        border: none;
    }

    .stButton > button:hover {
        background-color: #1A2A45;
    }

    .response-container {
        background-color: #f8f9fa;
        padding: 15px;
        border-radius: 10px;
        border-left: 4px solid #213354;
        margin: 10px 0;
    }

    .error-container {
        background-color: #fee;
        padding: 15px;
        border-radius: 10px;
        border-left: 4px solid #dc3545;
        margin: 10px 0;
    }
</style>
""", unsafe_allow_html=True)

# Inicialización del estado de sesión
if 'respuesta_actual' not in st.session_state:
    st.session_state.respuesta_actual = ""
if 'modo_edicion' not in st.session_state:
    st.session_state.modo_edicion = False

def responder_con_meta(pregunta_usuario):
    """
    Función principal que procesa la pregunta del usuario y genera respuesta con metadatos
    """
    try:
        # Buscar fragmentos similares en la base de datos vectorial
        fragmentos = db.similarity_search(pregunta_usuario, k=1)
        if not fragmentos:
            return "❌ No se encontró contexto relevante.", False

        texto = fragmentos[0].page_content
        doc_meta = None

        # Buscar metadatos del documento
        for doc in documents:
            if texto.strip()[:50] in doc["content"]:
                doc_meta = doc["meta"]
                break

        # Generar respuesta con Mistral
        respuesta = preguntar_a_mistral(pregunta_usuario)

        # Formatear respuesta con metadatos
        if doc_meta:
            salida = (
                f"📄 **Título:** {doc_meta['title']}\n"
                f"🗣️ **Lengua:** {doc_meta['lengua']}\n"
                f"📍 **Ubicación:** {doc_meta['ubicacion']}\n\n"
                f"🧠 **Respuesta:** {respuesta}"
            )
        else:
            salida = f"🧠 **Respuesta:** {respuesta}"

        return salida, True
    except Exception as e:
        return f"❌ **Error:** {str(e)}", False

def exportar_a_excel(texto):
    """
    Exporta la respuesta a un archivo Excel
    """
    df = pd.DataFrame([{"Respuesta": texto}])

    # Crear buffer en memoria
    buffer = io.BytesIO()

    # Escribir Excel al buffer
    with pd.ExcelWriter(buffer, engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name='Respuesta')

    # Obtener datos del buffer
    buffer.seek(0)

    return buffer.getvalue()

# Título y subtítulo
st.markdown('<div class="custom-title">Consulta RAG sobre Pueblos y Nacionalidades</div>', unsafe_allow_html=True)
st.markdown('<div class="subtitulo">💬 Pregúntale a Mistral sobre los artículos procesados.</div>', unsafe_allow_html=True)

# Espacio
st.markdown("<br>", unsafe_allow_html=True)

# Input para la pregunta
pregunta = st.text_area(
    "Escribe tu pregunta aquí:",
    height=100,
    placeholder="Ejemplo: ¿Cuáles son las características principales de la cultura Shuar?"
)

# Botones de acción
col1, col2, col3 = st.columns([2, 1, 1])

with col1:
    enviar = st.button("🔍 Enviar consulta", type="primary")

with col2:
    editar = st.button("✏️ Editar respuesta")

with col3:
    exportar = st.button("📤 Exportar Excel")

# Procesamiento de la consulta
if enviar and pregunta.strip():
    with st.spinner("Procesando consulta..."):
        respuesta, exito = responder_con_meta(pregunta)
        st.session_state.respuesta_actual = respuesta
        st.session_state.modo_edicion = False

# Toggle modo edición
if editar and st.session_state.respuesta_actual:
    st.session_state.modo_edicion = not st.session_state.modo_edicion

# Mostrar respuesta
if st.session_state.respuesta_actual:
    st.markdown("### Respuesta:")

    if st.session_state.modo_edicion:
        # Modo edición
        respuesta_editada = st.text_area(
            "Editar respuesta:",
            value=st.session_state.respuesta_actual,
            height=300,
            key="editor"
        )

        # Botones para guardar o cancelar edición
        col_save, col_cancel = st.columns([1, 1])
        with col_save:
            if st.button("💾 Guardar cambios"):
                st.session_state.respuesta_actual = respuesta_editada
                st.session_state.modo_edicion = False
                st.rerun()

        with col_cancel:
            if st.button("❌ Cancelar"):
                st.session_state.modo_edicion = False
                st.rerun()
    else:
        # Modo visualización
        if "❌" in st.session_state.respuesta_actual:
            st.markdown(f'<div class="error-container">{st.session_state.respuesta_actual}</div>', unsafe_allow_html=True)
        else:
            st.markdown(f'<div class="response-container">{st.session_state.respuesta_actual}</div>', unsafe_allow_html=True)

# Exportar a Excel
if exportar and st.session_state.respuesta_actual:
    try:
        # Generar archivo Excel
        excel_data = exportar_a_excel(st.session_state.respuesta_actual)

        # Nombre del archivo con timestamp
        nombre_archivo = f"respuesta_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"

        # Botón de descarga
        st.download_button(
            label="⬇️ Descargar archivo Excel",
            data=excel_data,
            file_name=nombre_archivo,
            mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
        )

        st.success("✅ Archivo Excel generado exitosamente!")

    except Exception as e:
        st.error(f"❌ Error al generar Excel: {str(e)}")

# Información adicional en sidebar
with st.sidebar:
    st.markdown("### 📋 Información")
    st.markdown("""
    **Funcionalidades:**
    - 🔍 Consulta documentos con RAG
    - ✏️ Edición de respuestas
    - 📤 Exportación a Excel
    - 🧠 Respuestas generadas por Mistral
    """)

    if st.session_state.respuesta_actual:
        st.markdown("### 📊 Estadísticas")
        palabras = len(st.session_state.respuesta_actual.split())
        caracteres = len(st.session_state.respuesta_actual)
        st.metric("Palabras", palabras)
        st.metric("Caracteres", caracteres)

# Footer
st.markdown("---")
st.markdown(
    "<div style='text-align: center; color: #666; font-size: 14px;'>"
    "Powered by Streamlit & Mistral AI"
    "</div>",
    unsafe_allow_html=True
)

Overwriting app.py


In [15]:
!pip install pyngrok

In [17]:
!npm install -g localtunnel
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0 &
!lt --port 8501

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴
added 22 packages in 2s
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴


  You can now view your Streamlit app in your browser.

  URL: http://0.0.0.0:8501

  Stopping...
your url is: https://modern-taxis-ask.loca.lt
/tools/node/lib/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:29987 (check your firewall settings)
    at Socket.<anonymous> (/tools/node/lib/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:524:28)
    at emitErrorNT (node:internal/streams/destroy:169:8)
    at emitErrorCloseNT (node:internal/streams/destroy:128:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:82:21)

Node.js v20.19.0


In [13]:
!python -m streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.180.165:8501

  Stopping...
  Stopping...


In [5]:
import gradio as gr
import pandas as pd
from datetime import datetime

respuesta_actual = ""

def responder_gradio_con_meta(pregunta_usuario):
    global respuesta_actual
    # Simulamos la respuesta (reemplaza esto con tu lógica real)
    doc_meta = {
        'title': "Saberes Ancestrales",
        'lengua': "Kichwa",
        'ubicacion': "Sierra Centro"
    }
    respuesta = "Los saberes ancestrales son transmitidos oralmente por generaciones."

    salida = (
        f"📄 **Título:** {doc_meta['title']}\n"
        f"🗣️ **Lengua:** {doc_meta['lengua']}\n"
        f"📍 **Ubicación:** {doc_meta['ubicacion']}\n\n"
        f"🧠 **Respuesta:** {respuesta}"
    )
    respuesta_actual = salida
    return salida

def exportar_excel():
    global respuesta_actual
    # ✅ Guardamos el archivo en la ruta correcta
    nombre_archivo = f"/mnt/data/respuesta_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
    df = pd.DataFrame([{"Respuesta": respuesta_actual}])
    df.to_excel(nombre_archivo, index=False)
    return nombre_archivo  # ← Esto sí lo muestra Gradio para descargar

css_personalizado = """
body, html {
    background-color: #111827 !important;
    color: #FFFFFF !important;
    font-family: 'Segoe UI', sans-serif;
}
textarea {
    background-color: #1f2937 !important;
    color: white !important;
    border-radius: 8px;
}
button {
    background-color: #1E3A8A !important;
    color: white !important;
    font-weight: bold;
    border-radius: 8px;
    padding: 10px 20px;
}
button:hover {
    background-color: #374151 !important;
}
"""

with gr.Blocks(css=css_personalizado) as demo:
    gr.Markdown("## 📚 Consulta RAG sobre Pueblos y Nacionalidades")

    pregunta = gr.Textbox(label="✍️ Escribe tu pregunta aquí", lines=3)
    respuesta = gr.Textbox(label="🧠 Respuesta con metadatos", lines=10, interactive=True)

    with gr.Row():
        boton = gr.Button("🔍 Enviar consulta")
        editar = gr.Button("✏️ Editar respuesta")
        exportar = gr.Button("📤 Exportar a Excel")

    archivo_generado = gr.File(label="📁 Archivo generado")

    boton.click(fn=responder_gradio_con_meta, inputs=pregunta, outputs=respuesta)
    editar.click(fn=lambda x: x, inputs=respuesta, outputs=respuesta)
    exportar.click(fn=exportar_excel, outputs=archivo_generado)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3475ad6f970e9d46be.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
